In [2]:
""" This program takes input from the scores database table (currently csv, later sql).
 Given a list of player IDs, it generates tables that attempt to make 4 or 5
 player tables of people who have played each other the least."""
#Uploaded to Github 8/10/15

import numpy as np
import pandas as pd
from pandas import Series, DataFrame
#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns
from __future__ import division
import itertools
from itertools import combinations

todays_players = [40,6,23,26,15,98,1,10,55,16,56,97,96]

In [3]:
scores_table = pd.read_csv('MahjongScores.csv')

In [4]:
scores_table.head()

,ScoreId,GameId,PlayerId,Rank,RawScore,Score,Unnamed: 6
0,1,1,23,1,0,38,NaN
1,2,1,52,2,0,13,NaN
2,3,1,6,3,0,-9,NaN
3,4,1,98,4,0,-42,NaN
4,5,2,6,1,0,30,NaN


In [234]:
scores_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2524 entries, 0 to 2523
Data columns (total 7 columns):
ScoreId       2524 non-null int64
GameId        2524 non-null int64
PlayerId      2524 non-null int64
Rank          2524 non-null int64
RawScore      2524 non-null int64
Score         2524 non-null int64
PlayerId_0    2524 non-null int64
dtypes: int64(7)
memory usage: 157.8 KB


In [235]:
#Check index
print scores_table['PlayerId'].min()
print scores_table['PlayerId'].max()

1
98


In [6]:
# 6th column is extra, drop it
scores_table.drop([scores_table.columns[6]], axis=1, inplace=True) # Note: zero indexed

In [7]:
scores_table.head()

,ScoreId,GameId,PlayerId,Rank,RawScore,Score
0,1,1,23,1,0,38
1,2,1,52,2,0,13
2,3,1,6,3,0,-9
3,4,1,98,4,0,-42
4,5,2,6,1,0,30


In [239]:
relevant_dict = {} # Dictionary to relate playerIDs to games 
      
# For each of today's players
for i in todays_players:
    games_rows = scores_table.loc[(scores_table['PlayerId'] == i)] # Pulling all of their rows
    for j in range(0,games_rows.shape[0]): # Iterate through those rows
        gameid = games_rows['GameId'].iloc[j] # Make a gameid for relevant game
        if gameid not in relevant_dict.keys(): # Check for gameid key in relevant_dict
            relevant_dict[gameid] = [] # Initialize list in dict if not already done
        relevant_dict[gameid].append(i) # Add playerID to gameid key in relevant_dict
        
games_df = scores_table.loc[(scores_table['GameId'].isin(relevant_dict.keys()))] # Pull all players from these games
games_df.head()

,ScoreId,GameId,PlayerId,Rank,RawScore,Score,PlayerId_0
0,1,1,23,1,0,38,22
1,2,1,52,2,0,13,51
2,3,1,6,3,0,-9,5
3,4,1,98,4,0,-42,97
4,5,2,6,1,0,30,5


In [224]:
# Create zeros dataframe of (#players,#players) size for matchups
num_players = len(todays_players)
matchups_df = DataFrame(np.zeros(num_players**2).reshape(num_players,num_players))
matchups_df.index = todays_players
matchups_df.columns = todays_players

In [225]:
# Populate matchups table

for gameid in uniquegames: # For each game
    gameid_players = [] # Reset players in the game
    relevant_rows = games_df.loc[(games_df['GameId'] == gameid)] # Pull the row for each player in that game
    gameid_players = relevant_rows['PlayerId'].tolist() # Put players from relevant rows into a list
    for player_x in gameid_players: # For each player in that game
        if player_x in todays_players: # If the player is playing today
            for player_y in gameid_players: # Look for opponent (may be self)
                if player_y in todays_players: # If second player is playing today
                    matchups_df.loc[player_x].loc[player_y] += 1 # Add 1 to the column that player is in
                
# Diagonals should be total # of games played for that player
matchups_df

,40,6,23,26,15,98,1,10,55,16,56,97,96
40,1,0,0,0,0,0,0,0,0,0,0,0,0
6,0,68,21,0,0,22,1,0,8,1,0,0,0
23,0,21,125,1,0,48,0,1,18,2,3,0,1
26,0,0,1,4,0,1,0,0,0,0,0,0,0
15,0,0,0,0,1,0,0,0,0,0,0,0,0
98,0,22,48,1,0,163,1,1,21,0,6,0,0
1,0,1,0,0,0,1,11,1,3,0,1,0,0
10,0,0,1,0,0,1,1,11,2,0,1,0,0
55,0,8,18,0,0,21,3,2,100,1,7,0,0
16,0,1,2,0,0,0,0,0,1,6,0,0,0


In [226]:
for i in range(0,matchups_df.shape[0]):
    for j in range(0,matchups_df.shape[1]):
        if (i != j) & (j > i): #Convert numbers above diagonal to percentages
            matchups_df.iloc[i].iloc[j] = (matchups_df.iloc[i].iloc[j] / matchups_df.iloc[i].iloc[i]) * 100

In [227]:
matchups_df

,40,6,23,26,15,98,1,10,55,16,56,97,96
40,1,0,0.000000,0.0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000
6,0,68,30.882353,0.0,0,32.352941,1.470588,0.000000,11.764706,1.470588,0.000000,0,0.000
23,0,21,125.000000,0.8,0,38.400000,0.000000,0.800000,14.400000,1.600000,2.400000,0,0.800
26,0,0,1.000000,4.0,0,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000
15,0,0,0.000000,0.0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000
98,0,22,48.000000,1.0,0,163.000000,0.613497,0.613497,12.883436,0.000000,3.680982,0,0.000
1,0,1,0.000000,0.0,0,1.000000,11.000000,9.090909,27.272727,0.000000,9.090909,0,0.000
10,0,0,1.000000,0.0,0,1.000000,1.000000,11.000000,18.181818,0.000000,9.090909,0,0.000
55,0,8,18.000000,0.0,0,21.000000,3.000000,2.000000,100.000000,1.000000,7.000000,0,0.000
16,0,1,2.000000,0.0,0,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,0,0.000


In [228]:
# Determine types of tables for today (whether there are 5 person tables or not)
if num_players < 8:
    total_tables = 1
else:
    table_det = num_players%4
    if table_det == 0:
        tables_4p = num_players/4
        tables_5p = 0
    elif table_det == 1:
        tables_4p = (num_players-1)/4
        tables_5p = 1
    elif table_det == 2:
        tables_4p = (num_players-2)/4
        tables_5p = 2
    elif table_det == 3:
        tables_4p = (num_players-3)/4
        tables_5p = 3
    total_tables = tables_4p + tables_5p

In [232]:
tables = []
tables_5p_made = 0
table_sums = []
temp_playerlist = list(todays_players) # Make a copy of today's players to use

while (len(temp_playerlist) > (4 + tables_5p)):
    # Generated list of combinations
    if tables_5p_made < tables_5p:
        table_combs = list(itertools.combinations(temp_playerlist,5))
        tables_5p_made += 1
    else:
        table_combs = list(itertools.combinations(temp_playerlist,4))

    # Calculate table matchmaking rating (lower means players have fewer games together)
    for table in table_combs:
        table_mm = 0
        for i in table:
            for j in table:
                if (j > i):
                    table_mm += matchups_df.loc[i].loc[j]
        table_sums.append(table_mm) # table_sums and table_combs have the same indexing

    # Remove entries with perfect ratings (table_sum=0)
    if 0 in table_sums:
        i = 0
        for table_sum in table_sums: # Iterate through sums to get index of perfect rating
            if table_sum == 0:
                break
            i += 1
        # Add current table to output tables
        tables.append(table_combs[i])
        for player in table_combs[i]: # Remove players from the pool that have already been seated
            temp_playerlist.remove(player)
    else:
        # Take first minimum value in list - gives priority matching to older players
        tables.append(table_combs(np.argmin(table_mm))) #np.argmin gives index of minimum value of list arg
        for player in table_combs[np.argmin(table_mm)]:
            temp_playerlist.remove(player)
            
# Clean up by making last table
if (len(temp_playerlist) == 4):
    tables.append(temp_playerlist)

In [233]:
tables

[(40, 6, 26, 15, 10), (98, 55, 16, 96), [23, 1, 56, 97]]